## Brio-Wu

Avec ce premier problème nous souhaitons tester les solveurs `hlld` et `fivewaves` de `fv2d`, et les comparer aux solveurs hlld et Roe de Athena++. Mis à part les solveurs, nous utiliserons une grille de 512 points et une CFL de 0,4. La reconstruction spatiale utilisée par défaut sera une _PLM_, combinée à un schéma temporel _Runge-Kutta_ d'ordre 2. On note que la divergence de $\vec{B}$ ne pose pas de problème en 1D et ne sera pas étudiée ici.

### Problem Set Up

In [ ]:
from datahandler import Fv2dCode, AthenaCode, IniFile
from pathlib import Path
home = Path().home() / "Documents"

In [ ]:
ini_file = IniFile(home/"fv2d/settings/mhd_sod_x.ini")
ini_file.set_param("mesh", "Nx", 512)
ini_file.set_param("solvers", "CFL", 0.4)
ini_file.set_param("physics", "gamma0", 2.0)

# HLLD solver + RK2
ini_file.set_param("solvers", "riemann_solver", "hlld")
ini_file.write("brio_wu.ini")

# HLLD solver + RK2 without div cleaning
ini_file.set_param("solvers", "riemann_solver", "hlld")
ini_file.set_param("solvers", "div_cleaning", "none")
ini_file.write("brio_wu_nodc.ini")

# Five waves solver + RK2 and div cleaning
ini_file.set_param("solvers", "riemann_solver", "fivewaves")
ini_file.write("brio_wu_5w.ini")

# Five waves solver + Euler time stepping and div cleaning
ini_file.set_param("solvers", "riemann_solver", "fivewaves")
ini_file.set_param("solvers", "time_stepping", "euler")
ini_file.write("brio_wu_5w_euler.ini")

# HLLD solver + Euler time stepping
ini_file.set_param("solvers", "riemann_solver", "hlld")
ini_file.set_param("solvers", "time_stepping", "euler")
ini_file.set_param("solvers", "div_cleaning", "none")
ini_file.write("brio_wu_euler.ini")

In [ ]:
# Compile and run the code
fv2d = Fv2dCode(base_path=home/"fv2d")
fv2d.compile()
fv2d.run(inifile="brio_wu.ini")
fv2d.run(inifile="brio_wu_nodc.ini")
fv2d.run(inifile="brio_wu_5w.ini")
fv2d.run(inifile="brio_wu_5w_euler.ini")
fv2d.run(inifile="brio_wu_euler.ini")


In [ ]:
athena = AthenaCode(home/"athena")
ini_file = IniFile(home/"athena/inputs/mhd/athinput.bw", athena_fmt=True)
ini_file.set_param("mesh", "nx1", 512)
ini_file.set_param("time", "cfl_number", 0.4)
ini_file.set_param("time", "tlim", 0.2)
ini_file.set_param("mesh", "x1max", 1.0)
ini_file.set_param("mesh", "x1min", 0.0)
ini_file.set_param("problem", "xshock", 0.5) # position de l'interface initiale
ini_file.write("athinput.bw")
athena.compile(ini_file)
athena.run(inifile="athinput.bw")

### Data Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datahandler import AthenaTabular, Fv2DH5
from pathlib import Path

In [ ]:
athena_data_path = Path("athena_output")
athena_data_file = list(sorted(athena_data_path.glob("Brio-Wu.*.tab")))[-1]
athena_data = AthenaTabular(athena_data_file)

In [ ]:
# hlld + RK2 Data
fv2d_data_path = Path("fv2d_output")
fv2d_data = Fv2DH5(fv2d_data_path/"run_brio_wu.h5", reshape=True)
fv2d_data_5w = Fv2DH5(fv2d_data_path/"run_brio_wu_5w.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x
fv2d_data_5we = Fv2DH5(fv2d_data_path/"run_brio_wu_5w_euler.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x
fv2d_data_e = Fv2DH5(fv2d_data_path/"run_brio_wu_euler.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x
fv2d_data_nodc = Fv2DH5(fv2d_data_path/"run_brio_wu_nodc.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x

In [ ]:
def plot_attrs(attr, *, longname=None, title="Brio-Wu Test") -> None:
    """ Plot the given attribute for all the data sets.
    Parameters
    ----------
    attr : str
        The attribute to plot.
    longname : str
        The long name of the attribute to plot, used for the $y$-axis.
        If None, the attribute name will be used.
    """
    plt.figure(figsize=(10, 10))
    plt.plot(athena_data.x, getattr(athena_data, attr), '--k', label="Athena-hlld")
    plt.plot(fv2d_data.x, getattr(fv2d_data, attr), '--', label="FV2D-hlld-RK2")
    plt.plot(fv2d_data_5w.x, getattr(fv2d_data_5w, attr), '--', label="FV2D-5waves-RK2")
    plt.plot(fv2d_data_5we.x, getattr(fv2d_data_5we, attr), '--', label="FV2D-5waves-Euler")
    plt.plot(fv2d_data_e.x, getattr(fv2d_data_e, attr), '--', label="FV2D-hlld-Euler")
    plt.plot(fv2d_data_nodc.x, getattr(fv2d_data_nodc, attr), '--', label="FV2D-hlld-noDC")
    plt.title(title)
    if longname is not None:
        plt.ylabel(longname)
    else:
        plt.ylabel(attr)
    plt.xlabel("x")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_attrs("rho", longname=r"$\rho$")
plot_attrs("u", longname=r"$v_x$")
plot_attrs("v", longname=r"$v_y$")
plot_attrs("p", longname=r"$P$")
# plot_attrs("Bx", longname=r"$B_x$")
plot_attrs("By", longname=r"$B_y$")
# plot_attrs("Bz", longname=r"$B_z$")

### Error Analysis

In [ ]:
from typing import Callable


def absolute_error(y: np.ndarray, y_ref: np.ndarray) -> np.ndarray:
    """ Compute the absolute error between two arrays.
    Parameters
    ----------
    y : np.ndarray
        The data to compare.
    y_ref : np.ndarray
        The reference data.
    Returns
    -------
    np.ndarray
        The absolute error between the two arrays.
    """
    return np.abs(y - y_ref)


def relative_error(y: np.ndarray, y_ref: np.ndarray, norm_ord=1) -> np.ndarray:
    """ Compute the relative error between two arrays.
    Parameters
    ----------
    y : np.ndarray
        The data to compare.
    y_ref : np.ndarray
        The reference data.
    Returns
    -------
    np.ndarray
        The relative error between the two arrays.
    """
    return np.abs(y - y_ref) / np.abs(y_ref)

def plot_error(attr: str, *, longname: str=None, error_type: Callable = relative_error, norm_ord=2, scale='log') -> None:
    """ Plot the error between the data sets.
    Parameters
    ----------
    attr : str
        The attribute to plot.
    longname : str
        The long name of the attribute to plot, used for the $y$-axis.
        If None, the attribute name will be used.
    error_type : Callable
        The error function to use. Can be either absolute_error or relative_error.
    norm_ord : int
        The order of the norm to use. Default is 2.
    """

    plt.figure(figsize=(10, 10))
    e1 = error_type(getattr(fv2d_data, attr), getattr(athena_data, attr))
    e2 = error_type(getattr(fv2d_data_5w, attr), getattr(athena_data, attr))
    e3 = error_type(getattr(fv2d_data_5we, attr), getattr(athena_data, attr))
    e4 = error_type(getattr(fv2d_data_e, attr), getattr(athena_data, attr))
    e5 = error_type(getattr(fv2d_data_nodc, attr), getattr(athena_data, attr))
    plt.plot(fv2d_data.x, e1, '--', label=f"FV2D-hlld-RK2, L{norm_ord}-norm= {np.linalg.norm(e1, ord=norm_ord)/len(e1):.2e}")
    plt.plot(fv2d_data.x, e2, '--', label=f"FV2D-5waves-RK2, L{norm_ord}-norm= {np.linalg.norm(e2, ord=norm_ord)/len(e2):.2e}")
    plt.plot(fv2d_data.x, e3, '--', label=f"FV2D-5waves-Euler, L{norm_ord}-norm= {np.linalg.norm(e3, ord=norm_ord)/len(e3):.2e}")
    plt.plot(fv2d_data.x, e4, '--', label=f"FV2D-hlld-Euler, L{norm_ord}-norm= {np.linalg.norm(e4, ord=norm_ord)/len(e4):.2e}")
    plt.plot(fv2d_data.x, e5, '--', label=f"FV2D-hlld-noDC, L{norm_ord}-norm= {np.linalg.norm(e5, ord=norm_ord)/len(e5):.2e}")
    plt.title("Brio-Wu Test")
    plt.ylim(1e-6, 1e0)
    if longname is not None:
        plt.ylabel(longname)
    else:
        plt.ylabel(attr)
    plt.xlabel("x")
    plt.yscale(scale)
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
plot_error("rho", longname=r"$\rho$", error_type=absolute_error)
plot_error("u", longname=r"$v_x$", error_type=absolute_error)
plot_error("v", longname=r"$v_y$", error_type=absolute_error)
plot_error("p", longname=r"$P$", error_type=absolute_error)
plot_error("By", longname=r"$B_y$", error_type=absolute_error)

### Convergence Analysis

Le but est ici de tester la convergence de nos différents solveurs selon la taille du maillage, i.e. le nombre de cellules. Nous choissisons de tester chaque solveurs pour les valeurs suivantes $Nx=32, 64, 128 , 256, 512$

In [ ]:
from datahandler import Fv2dCode, IniFile, AthenaCode
from pathlib import Path
home = Path().home() / "Documents"

In [ ]:
# generate all the ini_files for the different tests
original_ini = home/"fv2d/settings/mhd_sod_x.ini"
athena_ini = home/"athena/inputs/mhd/athinput.bw"
for solver in ["hlld", "fivewaves"]:
    for time_stepping in ["RK2", "euler"]:
        for i in range(5, 10):
            new_ini = f"brio_wu_{2**i:04d}_{solver}_{time_stepping}.ini"
            ini = IniFile(original_ini)
            ini.set_param("mesh", "Nx", 2**i)
            ini.set_param("solvers", "riemann_solver", solver)
            ini.set_param("solvers", "time_stepping", time_stepping)
            ini.set_param("solvers", "div_cleaning", "none")
            ini.set_param("solvers", "CFL", 0.4)
            ini.set_param("physics", "gamma0", 2.0)
            ini.write(new_ini)
            at_ini = IniFile(athena_ini, athena_fmt=True)
            at_ini.set_param("mesh", "nx1", 2**i)
            at_ini.set_param("time", "cfl_number", 0.4)
            at_ini.set_param("time", "tlim", 0.2)
            at_ini.write(f"athinput_{2**i:04d}.bw")

In [ ]:
# Here we assume (we know) that the fv2d code is already compiled
# and that the athena code is already compiled
ath_output = Path('athena_output')
for item in Path().glob("athinput_*.bw"):
    athena = AthenaCode(home/"athena")
    this_output = ath_output / item.stem.replace("athinput_", "")
    this_output.mkdir(exist_ok=True, parents=True)
    athena.run(inifile=item.name, destination=this_output)

In [ ]:
fv2d = Fv2dCode(base_path=home/"fv2d")
# fv2d.compile()
fv2d_output = Path('fv2d_output')
with open('fv2d.log', 'w') as f:
    f.write("FV2D runs\n")

files = list(sorted(Path().glob("brio_wu_*.ini")))
for i, item in enumerate(files):
    with open('fv2d.log', 'a') as f:
        f.write(f"[{i+1}/{len(files)}] - Run {item.name} started\n")
    nx = item.stem.split("_")[2]
    this_output = fv2d_output / nx
    this_output.mkdir(exist_ok=True, parents=True)
    fv2d.run(inifile=item.name, destination=this_output)
    with open('fv2d.log', 'a') as f:
        f.write(f"[{i+1}/{len(files)}] - Run {item.name} finished\n")

In [ ]:
from datahandler import Fv2DH5, AthenaTabular
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
athena_data = {}
fv2d_data = {}
errors = {"-".join(algo.split('_')[:-1]): [] for algo in fv2d_data}
for i in range(5, 10):
    dirname = Path("athena_output") / f"{2**i:04d}"
    item = list(sorted(dirname.glob("Brio-Wu.*.tab")))[-1]
    athena_data[f"{2**i:04d}"] = AthenaTabular(item)
    athena_data[f"{2**i:04d}"].rescale() # by default on AThena is [-0.5, 0.5] and on fv2d is [0, 1]
    item = list(sorted(Path(f"fv2d_output/{2**i:04d}").glob("run_brio_wu_*.h5")))
    fv2d_data[f"{2**i:04d}"] = [(str(it.stem), Fv2DH5(it, reshape=True)) for it in item]

In [ ]:
errors = {"-".join(algo[0].split('_')[-2:]):[] for algo in list(fv2d_data.values())[0]}
for i in range(5, 10):
    nx = f"{2**i:04d}"
    for algo, val in fv2d_data[nx]:
        algo = "-".join(algo.split('_')[-2:]) # on veut rassembler les algo
        err = np.linalg.norm(val.rho - athena_data[nx].rho, ord=2)/int(nx)
        errors[algo].append(err)

In [ ]:
plt.figure(figsize=(10, 10))
nx_values = np.array([2**i for i in range(5, 10)])
for algo, err in errors.items():
    m, b = np.polyfit(np.log(nx_values), np.log(err), deg=1)
    plt.plot(nx_values, err, '--o', label=fr"{algo}: $\log(err)={m:.2f}N_x{b:.2f}$")
plt.title("Brio-Wu Test: Error Convergence")

O_x = errors["fivewaves-RK2"][0] * (nx_values[0] / nx_values)
O_x2 = errors["fivewaves-RK2"][0] * (nx_values[0] / nx_values)**2  
plt.plot(nx_values, O_x, ':', label=r"$\mathcal{O}(\Delta x)$")
plt.plot(nx_values, O_x2, ':', label=r"$\mathcal{O}(\Delta x^2)$")

plt.text(nx_values[-1], O_x2[-1], r"$\Delta x^2$", fontsize=12)
plt.text(nx_values[-1], O_x[-1], r"$\Delta x$", fontsize=12)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r"$N_x$")
plt.ylabel(r"$L^2$ error")
plt.legend()
plt.grid()
plt.show()